In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tiurin/projects/ExampleProject/Temp/code/eye_quality/')

In [76]:
#|export
def get_confusion_matrix(predictions, targets):
    confusion_mat = np.zeros((3, 3), dtype=int)
    
    for i in range(targets.shape[0]):
        confusion_mat[targets[i], predictions[i]] += 1
    return confusion_mat

In [77]:
#|export
import numpy as np
import dataset_utils
def get_confusion_matrix_image(model, dataloader, device):
    confusion_mat = np.zeros((3, 3), dtype=int)
    model = model.to(device)

    confusion_images = [[None, None, None] for i in range(3)]

    for batch in dataloader:
        classes, imgs = batch
        imgs = imgs.to(device)
        res = model.predict(imgs)

        for i in range(classes.shape[0]):
            confusion_mat[classes[i], res[i]] += 1
            if confusion_images[classes[i]][res[i]] is None:
                confusion_images[classes[i]][res[i]] = dataset_utils.convert_to_image(imgs[i])
    return confusion_mat, confusion_images

In [70]:
#|export
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, class_names, normalize=False):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    #plt.ioff()
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    if normalize:
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.ion()
    return figure

In [71]:
#|export
import io
def plot_to_image(figure):
    """
    Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call.
    """
    plt.close(figure)
    figure.canvas.draw()
    image = np.frombuffer(figure.canvas.tostring_rgb(), dtype=np.uint8)
    image = image.reshape(figure.canvas.get_width_height()[::-1] + (3,))
    image = image.transpose(2,0,1)
    
    return image

In [50]:
#|export
def precision(confusionMat):
    nbClasses = confusionMat.shape[0]
    averagePrecision = 0

    for idx in range(nbClasses):
        averagePrecision += confusionMat[idx,idx]/np.sum(confusionMat[:,idx])
    averagePrecision /= nbClasses
    
    return averagePrecision

In [51]:
#|export
def recall(confusionMat):
    nbClasses = confusionMat.shape[0]
    averageRecall = 0

    for idx in range(nbClasses):
        averageRecall += confusionMat[idx,idx]/np.sum(confusionMat[idx,:])
    averageRecall /= nbClasses

    return averageRecall

In [52]:
#|export
def accuracy(confusion_mat):
    accuracy = confusion_mat.trace()/confusion_mat.sum()
    return accuracy

In [53]:
#|export
def f1_score(confusion_mat):
    precision_val = precision(confusion_mat)
    recall_val = recall(confusion_mat)
    f1_score = 2*(precision_val*recall_val)/(precision_val+recall_val)
    return f1_score

In [54]:
#|export
def class_accuracy(confusion_mat, class_num):
    return confusion_mat[class_num, class_num]/confusion_mat[class_num,:].sum()

In [74]:
#|export
import torch
import torchvision
import math
def make_image_grid(imgs, row_size=3):
    num_imgs = imgs.shape[0] if isinstance(imgs, torch.Tensor) else len(imgs)
    is_int = imgs.dtype==torch.int32 if isinstance(imgs, torch.Tensor) else imgs[0].dtype==torch.int32
    nrow = min(num_imgs, row_size)
    ncol = int(math.ceil(num_imgs/nrow))
    imgs = torchvision.utils.make_grid([torch.from_numpy(i) for i in imgs], nrow=nrow, pad_value=128 if is_int else 0.5)
    imgs = torch.clamp(imgs, min=0, max=255)
    np_imgs = imgs.cpu().numpy()
    np_imgs = np_imgs.astype('uint8')
    return np_imgs

In [2]:
#|default_exp evaluation_utils
#|hide
nbpath = '/home/tiurin/projects/ExampleProject/Temp/code/nbs/evaluation.ipynb'
import nbdev; nbdev.nbdev_export(nbpath)